In [ ]:
# modules
import pandas as pd
import numpy as np
from datetime import datetime
import pickle
import os
import glob
from concurrent import futures           

In [ ]:
TOKENS = ['FRAX','EURS','XCHF','XAUt','FEI','GUSD','BUSD','sUSD']
DECIMALS = [18,2,18,6,18,2,18,18]
FIRST_BLOCKS = [11465581,5835474,6622987,9339031,12168368,6302486,8523552,5767935]
LAST_BLOCK = 14497033

In [ ]:
# calculates the holding time for all coins contained in a transaction

def holdingTimeCalc(file):

    """
    1. loads the address slices
    2. for every address (excluding 0x0-address)
        2.1 loads all spending transactions of an address
        2.2 for each spending transaction
            2.2.1 loads block of transaction
            2.2.2 initializes holding time and weights list
            2.2.3 applies LIFO-method to calculate the holding time of different tokens contained in a transaction
            2.2.4 safes it in a dictionary
    """

    print('{}: initialized'.format(file))
    addresses = pd.read_pickle(file)
    addresses = list(addresses.keys())

    # fromTrxDict = df.groupby('from_address')['transaction_hash_unique'].apply(list).to_dict() # transaction dictionary
    # fromTrxIndex = df.groupby('from_address')['index'].apply(list).to_dict()
    # toTrxDict = df.groupby('to_address')['transaction_hash_unique'].apply(list).to_dict()
    # toTrxIndex = df.groupby('to_address')['index'].apply(list).to_dict()
    trxValueOrig = dict(zip(df['transaction_hash_unique'], df['value']))
    trxBlock = dict(zip(df['transaction_hash_unique'], df['block_number']))

    holdingTimeDict = {}

    for address in addresses: 
        if address != '0x0000000000000000000000000000000000000000':
            fromTrxList = list(fromTrxDict[address]) # list of all spending transactions
            trxValue = trxValueOrig.copy() 
            for fromTrx in fromTrxList:
                block = trxBlock[fromTrx]
                fromTransactions = dict(zip(fromTrxDict[address],fromTrxIndex[address]))
                toTransactions = dict(zip(toTrxDict[address],toTrxIndex[address]))
                index = fromTransactions[fromTrx]
                transactionList = [v for v in toTrxDict[address] if toTransactions[v] < index] # consider only past receiving transactions
                fromValue = trxValue[fromTrx]
                weights = []
                holdingTime = []

                for i in reversed(range(len(transactionList))): # itter through receiving transactions in reverse order
                    toTrx = transactionList[i]
                    if int(fromValue) == 0:
                        break
                    else:   
                        if int(trxValue[toTrx]) >= int(fromValue):
                            if int(fromValue) > 0:
                                weights.append(int(fromValue))
                                holdingTime.append(block - trxBlock[toTrx]) # holding time in number of blocks between trx
                            trxValue[toTrx] = str(int(trxValue[toTrx]) - int(fromValue))
                            fromValue = 0
                        else: 
                            if int(fromValue) > 0:
                                weights.append(int(trxValue[toTrx]))
                                holdingTime.append(block - trxBlock[toTrx])
                            fromValue = str(int(fromValue) - int(trxValue[toTrx]))
                            trxValue[toTrx] = 0  

                ht = [holdingTime[x] for x in range(len(holdingTime)) if weights[x] > 0]
                w = [weights[x] for x in range(len(weights)) if weights[x] > 0]
                if sum(ht) > 0:
                    holdingTimeDict[fromTrx] = [w, ht]

    f = open("holdingTime_{}".format(file),"wb")
    pickle.dump(holdingTimeDict,f)
    f.close()  
    print('{}: sucess'.format(file))

for i in range(len(TOKENS)):
    TOKEN = TOKENS[i]
    FIRST_BLOCK = FIRST_BLOCKS[i]
    
    # load the test transaction data
    TransactionsFull = pd.read_csv('{}_token_transfers.csv'.format(TOKEN))

    # load the supply data (supply at the last block per day - from etherscan)
    Supply = pd.read_csv('{}_supply.csv'.format(TOKEN))

    # data cleaning
    df = TransactionsFull.sort_values(by=['block_number', 'log_index']).reset_index(drop=True)
    
    # exclude transactions where value is 0 or from_address == to_address
    df = pd.DataFrame(df.loc[((df['value'] != 0) | (df['value'] != '0')) & (df['from_address'] != df['to_address'])])

    # create new transaction_hash column by appending log_index to make every hash unique
    transactionHash = df.transaction_hash.to_list()
    logIndex = df.log_index.to_list()
    transactionHashUnique = []
    for i in range(0,len(transactionHash)):
        t = transactionHash[i]
        l = logIndex[i]
        u = str(t)+'__'+str(l)
        transactionHashUnique.append(u)

    df['transaction_hash_unique'] = transactionHashUnique
    df['index'] = [x for x in range(len(df))]
    df.to_csv('{}_token_transfers_new'.format(TOKEN))
    fromTrxDict = df.groupby('from_address')['transaction_hash_unique'].apply(list).to_dict()
    fromTrxIndex = df.groupby('from_address')['index'].apply(list).to_dict()
    toTrxDict = df.groupby('to_address')['transaction_hash_unique'].apply(list).to_dict()
    toTrxIndex = df.groupby('to_address')['index'].apply(list).to_dict()

    addresses = list(fromTrxDict.keys())
    c = 1
    for a in np.array_split(addresses, 40):
        a_subset = {key: value for key, value in fromTrxDict.items() if key in a}
        f = open("sliced_accounts_{}_{}.pkl".format(TOKEN,c),"wb")
        pickle.dump(a_subset,f)
        f.close() 
        c += 1
        
    with futures.ProcessPoolExecutor(max_workers=40) as ex:
        for slice in glob.glob('sliced_accounts_{}_*.pkl'.format(TOKEN)):
            ex.submit(holdingTimeCalc, slice)
   
    d0 = pd.read_pickle('holdingTime_sliced_accounts_{}_1.pkl'.format(TOKEN))
    for _filename in glob.glob('holdingTime_sliced_accounts_{}_*.pkl'.format(TOKEN)):
        d1 = pd.read_pickle(_filename)
        d0.update(d1) 

    f = open('{}_holdingTimes.pkl'.format(TOKEN),'wb')
    pickle.dump(d0,f)
    f.close()
    print('{}_holdingTimes.pkl: sucess'.format(TOKEN)) 

In [ ]:
# calculates the MicroVelocity from the holding time data

def microVelocityCalc(file):
    
    """
    1. reads in the sliced addresses
    2. for every address
        2.1 initiates 0-array of length (last block - first block)
        2.2 for every transaction
            2.2.1 calculates microVelocity
                    - sum for the comparison with Fisher's eq.
                    - mean for analysis
                    - snapshots at the last block per day for analysis
    """

    ts = pd.read_csv('block_timestamps_complete.csv', parse_dates=['timestamp'])
    ts = ts.loc[(ts.block_number >= FIRST_BLOCK) & (ts.block_number <= LAST_BLOCK)]
    ts_w = pd.DataFrame(ts[['timestamp','block_number']].groupby(pd.Grouper(key='timestamp', axis=0, freq='D')).first()).reset_index()

    first = pd.DataFrame(ts[['timestamp','block_number']].groupby(pd.Grouper(key='timestamp', axis=0, freq='D')).first()).reset_index()
    first = first['block_number'].to_list()
    last = pd.DataFrame(ts[['timestamp','block_number']].groupby(pd.Grouper(key='timestamp', axis=0, freq='D')).last()).reset_index()
    last = last['block_number'].to_list()

    trxBlock = dict(zip(df.transaction_hash_unique, df.block_number)) # dictionary with {transaction_hash : block_number}
    SupplyDict = dict(zip(Supply.block_number, Supply.supply)) # dictionary with {block_number : supply}
    holdingTime = pd.read_pickle('{}_holdingTimes.pkl'.format(TOKEN)) # dictionary with {transaction_hash : avg_ht}
    fromTrxDict = df.groupby('from_address')['transaction_hash_unique'].apply(list).to_dict()
    velocity_sum = {}
    velocity_mean = {}
    velocity_last = {}
    
    addresses = pd.read_pickle(file)
    addresses = list(addresses.keys())
    print('{}_initialized'.format(file))
    
    for address in addresses:
        ind_velocity = np.zeros(LAST_BLOCK+1)
        transactions = list(fromTrxDict[address])
        for trx in transactions: # take all transactions
            if trx in holdingTime.keys():
                am = holdingTime[trx][0] # number of coins with holding time tau
                ht = holdingTime[trx][1] # holding time in number of blocks
                if sum(ht) > 0:
                    height = trxBlock[trx] # block_number
                    for a in range(len(am)):
                        ht_ = ht[a]
                        am_ = am[a]
                        if ht_ > 0:
                            for t in range(int(round(height-ht_)),height): # loop over the blocks
                                ind_velocity[t] += int(am_) / ht_ * 1/int(SupplyDict[t])
        s = [sum(ind_velocity[first[i]:last[i]]) for i in range(len(first))]
        m = [s[i]/(last[i]-first[i]) for i in range(len(first))]
        l = [ind_velocity[i] for i in last] # snapshots at the last block per day
        velocity_sum[address] = s
        velocity_mean[address] = m 
        velocity_last[address] = l
        
    f = open("microVelocity_sum_{}".format(file),"wb")
    pickle.dump(velocity_sum,f)
    f.close()  
    f = open("microVelocity_mean_{}".format(file),"wb")
    pickle.dump(velocity_sum,f)
    f.close()  
    f = open("microVelocity_last_{}".format(file),"wb")
    pickle.dump(velocity_last,f)
    f.close()  
    print('{}: sucess'.format(file))

for i in range(len(TOKENS)):
    TOKEN = TOKENS[i]
    FIRST_BLOCK = FIRST_BLOCKS[i]
    
    # load the test transaction data
    TransactionsFull = pd.read_csv('{}_token_transfers_new.csv'.format(TOKEN))

    # load the supply data (supply at the last block per day - from etherscan)
    Supply = pd.read_csv('{}_supply.csv'.format(TOKEN))
    
    with futures.ProcessPoolExecutor(max_workers=40) as ex:
        for slice in glob.glob('sliced_accounts_{}_*.pkl'.format(TOKEN)):
            ex.submit(microVelocityCalc, slice)

    d0 = pd.read_pickle('microVelocity_sum_sliced_accounts_{}_1.pkl'.format(TOKEN))
    for file in glob.glob('microVelocity_sum_sliced_accounts_{}_*.pkl'.format(TOKEN)):
        d1 = pd.read_pickle(file)
        d0.update(d1) 

    f = open('{}_microVelocities_sum.pkl'.format(TOKEN),'wb')
    pickle.dump(d0,f)
    f.close()

    d0 = pd.read_pickle('microVelocity_mean_sliced_accounts_{}_1.pkl'.format(TOKEN))
    for file in glob.glob('microVelocity_mean_sliced_accounts_{}_*.pkl'.format(TOKEN)):
        d1 = pd.read_pickle(file)
        d0.update(d1) 

    f = open('{}_microVelocities_mean.pkl'.format(TOKEN),'wb')
    pickle.dump(d0,f)
    f.close()

    d0 = pd.read_pickle('microVelocity_last_sliced_accounts_{}_1.pkl'.format(TOKEN))
    for file in glob.glob('microVelocity_last_sliced_accounts_{}_*.pkl'.format(TOKEN)):
        d1 = pd.read_pickle(file)
        d0.update(d1) 

    f = open('{}_microVelocities_snapshots.pkl'.format(TOKEN),'wb')
    pickle.dump(d0,f)
    f.close()
    print('{}_microVelocities_snapshots: sucess'.format(TOKEN))

    for file in glob.glob('sliced_accounts_{}_*.pkl'.format(TOKEN)):
        os.remove(file)
    for file in glob.glob('microVelocity_sum_sliced_accounts_{}_*.pkl'.format(TOKEN)):
        os.remove(file)
    for file in glob.glob('microVelocity_mean_sliced_accounts_{}_*.pkl'.format(TOKEN)):
        os.remove(file)
    for file in glob.glob('microVelocity_last_sliced_accounts_{}_*.pkl'.format(TOKEN)):
        os.remove(file)

In [ ]:
# turns pickle file into data frame for convenience
ts = pd.read_csv('block_timestamps_complete.csv', parse_dates=['timestamp'])

for i in range(len(TOKENS)):
    TOKEN = TOKENS[i]
    FIRST_BLOCK = FIRST_BLOCKS[i]
    dct = pd.read_pickle('{}_microVelocities_last.pkl'.format(TOKEN))
    df = pd.DataFrame.from_dict(dct, orient='index').T
    df['total'] = df.sum(axis=1)
    ts2 = pd.DataFrame(ts.loc[(ts.block_number >= FIRST_BLOCK) & (ts.block_number <= LAST_BLOCK)])
    ts2 = pd.DataFrame(ts2[['timestamp','block_number']].groupby(pd.Grouper(key='timestamp', axis=0, freq='D')).last()).reset_index()
    timestamp = ts2.timestamp.to_list()
    df['timestamp'] = timestamp
    df.to_csv('{}_microVelocities_last.csv'.format(TOKEN),index=False)